In [ ]:
from pathlib import Path
import PySOMVis as PySomVisModule
from PySOMVis.pysomvis import PySOMVis
from PySOMVis.SOMToolBox_Parse import SOMToolBox_Parse
from tqdm import tqdm
from typing import List, Tuple
from minisom import MiniSom
import scipy
from scipy.spatial.distance import pdist, squareform
from skimage.transform import resize

import matplotlib.pyplot as plt
import seaborn as sns
# import somoclu
from random import randrange
import numpy as np
# import panel as pn


# import holoviews as hv
# hv.extension('bokeh')
#hv.extension('bokeh')
# import os

DATASET_PATH = Path(PySomVisModule.__file__).parent / 'datasets'

In [ ]:
a = np.arange(1)
b = np.arange(30)

In [ ]:
a

In [ ]:
b

In [ ]:
aa, bb = np.meshgrid(a, b)

In [ ]:
def gaussian(c, sigma):
    d = 2*sigma*sigma
    ax = np.exp(-np.power(aa-aa[c], 2)/d)
    ay = np.exp(-np.power(bb-bb[c], 2)/d)
    return (ax * ay).T  # the external product gives a matrix

In [ ]:
dist = (gaussian(0, 1.) / 10)[0]

In [ ]:
dist

In [ ]:
dist.shape

# Implementation Details

Aligned SOMs aims at training mulitple layers of n SOMs with differently weighted subsets of attributes.

The Alignd SOM implementation extends the well known MiniSom package.


## Layer Weighting

Two aspects or concepts of features in a dataset are differently weightet by different layers of the Alignd SOMs. The first layer uses a weighting ratio between aspect A and aspect B features of 1:0. The middle or center layer, weights both aspects equally. The last layer uses a weighting ratio of 0:1.

We create the weights by layer in te **AlignedSom** class using the method **_create_weights_by_layer**. The **AlignedSom** accepts a parameter **concept_indices** which has to be a boolean List inidcating if the feature belongs to aspect A (True or 1) or if it belongs to aspect B (False or 0). 

## Layer inizialization (Orientation / Codebook inizialization)

We create n SOM layers inizializing them  identically using the same common codebook but weighting them by the respective layer weight vector (from 0 for group A attributes and 1 for group B to 0/a for groups A and B in n steps. 

The inizialization of the layers in the **AlignedSom** class is done in the method **_create_layers**. We either crate the common codebook randomly or train the center SOM (trained with unweighted data) and use it as basis for all layer inizializations. This can be changeed by the parameter **initial_codebook_inizialization** ("random" or "pretrained).
The weighting of the layers is done by the **weights_by_layer** as explaind in the previous section. One Layer is represented by the **Layer** class which extends the MiniSom algorithm by overwriting the **update** method.

## Training

We train multiple layers of SOMs iteratively with an online training algorithm.
1) select a random layer and a random observation from the dataset
2) select the winning unit in the selected layer based on the weighted feature vector
3) train all layers updating the weights based on the same winning unint
    * the randomly selected layer is updated as in the normal SOM training
    * all other layers update the weights similarly but the margin of the update is based on the distance to the selected layer
    * all layers use the weighted feature vector based on their respective layer weights
4) iterate steps 1-3 N times

### Layer distances

The distance of the layers is defined as follows.
* the distance to the layer to iteslf is 1.0 -> normal SOM update rule
* the distance to the neighboring layer is a fraction (layer_distance_ratio) of the distance between neighbooring units in one layer (by default 1/10)
* the distance between the first and the last layer is the same distance as the distance of the upper left and lower right unit of each map.
* the distance between all other layers is a linear interploation of the previus two

In [ ]:
def db_file_string(dataset_name: str, extension: str) -> str:
    return str((DATASET_PATH / dataset_name / f'{dataset_name}.{extension}').resolve())

# load one of animals, iris, chainlink, 10clusters, BostonHousing
def load_dataset(name: str):
    input_data = SOMToolBox_Parse(db_file_string(name, 'vec')).read_weight_file()
    components = SOMToolBox_Parse(db_file_string(name, 'tv')).read_weight_file()
    weights = SOMToolBox_Parse(db_file_string(name, 'wgt.gz')).read_weight_file()
    classinfo = SOMToolBox_Parse(db_file_string(name, 'cls')).read_weight_file()
    return input_data, components, weights, classinfo

In [ ]:
from typing import Tuple
import numpy as np
from minisom import MiniSom


class Layer(MiniSom):
    def __init__(self,
                 dimension: Tuple[int, int],
                 input_len,
                 initial_codebook,
                 sigma=1,
                 learning_rate=0.5,
                 neighborhood_function='gaussian',
                 activation_distance='euclidean',
                 random_seed=None):
        super().__init__(
            x=dimension[0],
            y=dimension[1],
            input_len=input_len,
            sigma=sigma,
            learning_rate=learning_rate,
            neighborhood_function=neighborhood_function,
            topology='rectangular',
            activation_distance=activation_distance,
            random_seed=random_seed)

        self._weights = initial_codebook

    # changed update to include the distance to the layer in the neighborhood
    # todo: not sure if only winning unit updated or whole neighbourhood for other layers
    def update(self, x, win, layer_dist, t, max_iteration):
        eta = self._decay_function(self._learning_rate, t, max_iteration)
        # sigma and learning rate decrease with the same rule
        sig = self._decay_function(self._sigma, t, max_iteration)
        # improves the performances
        g = self.neighborhood(win, sig) * eta * layer_dist
        # w_new = eta * neighborhood_function * (x-w)
        self._weights += np.einsum('ij, ijk->ijk', g, x-self._weights)

In [ ]:
class AlignedSom(MiniSom):
    def __init__(self,
                 dimension: Tuple[int, int],
                 data: np.ndarray,  # 2d numpy array
                 concept_indices: List[bool],  # boolean list if feature belongs to concept A or concept B
                 num_layers: int = 100,
                 layer_distance_ratio: float = 0.1,
                 sigma: float = 1.0,
                 learning_rate: float = 0.5,
                 neighborhood_function: str = 'gaussian',
                 activation_distance: str = 'euclidean',
                 initial_codebook_inizialization: str = 'random',  # random or pretrained
                 random_seed=None):
        super().__init__(
            x=dimension[0],
            y=dimension[1],
            input_len=data.shape[1],
            sigma=sigma,
            learning_rate=learning_rate,
            neighborhood_function=neighborhood_function,
            topology='rectangular',
            activation_distance=activation_distance,
            random_seed=random_seed)
        self.data = data
        self.dimension = dimension
        self.concept_indices = concept_indices
        self.num_layers = num_layers
        self.layer_distance_ratio = layer_distance_ratio
        self._neighborhood_function = neighborhood_function
        self._initial_codebook_inizialization = initial_codebook_inizialization
        self.random_seed = random_seed

        self.weights_by_layer: np.ndarray = self._create_weights_by_layer()
        self.layers: List[Layer] = self._create_layers()
        self.layer_distances = self._create_layer_distances()

    def train(self,
              data: np.ndarray,  # 2d numpy array,
              num_iterations):
        n_observations = data.shape[0]
        for t in tqdm(range(num_iterations)):
            selected_layer = randrange(0, self.num_layers)
            selected_observation = randrange(0, n_observations)
            # print(f'selected layer: {selected_layer}')
            # print(f'selected observation: {selected_observation}')
            winner = self.layers[selected_layer].winner(
                data[selected_observation] * self.weights_by_layer[selected_layer])
            for i, layer in enumerate(self.layers):
                # print(f'current layer: {i}')
                # ĺayer_dist = self.layer_distance(t, num_iterations, np.abs(selected_layer - i))
                ĺayer_dist = self.layer_distances[np.abs(selected_layer - i)]
                # print(f'distance: {ĺayer_dist}')
                layer.update(data[selected_observation] * self.weights_by_layer[i],
                             winner,
                             ĺayer_dist,
                             t,
                             num_iterations)

    # the distance between one layer and the next is defined by the distance between neighboring units
    # multiplyed by some fraction "layer_distance_ratio"
    def layer_distance(self, t, max_iteration, grid_distance):
        if grid_distance == 0.0:
            return 1.0
        sig = self._decay_function(self._sigma, t, max_iteration)
        distance_neighboring_units = self.neighborhood((0, 0), sig)[(0, 1)]
        return distance_neighboring_units * (self.layer_distance_ratio / grid_distance)

    # return the codebook weights for all layers
    def get_layer_weights(self) -> List[np.ndarray]:
        return [layer.get_weights() for layer in self.layers]
    
    def _create_layer_distances(self):
        distance_matrix = self.neighborhood((0, 0), 1.0)
        distance_neighboring_units = distance_matrix[(0, 1)]
        distance_neighboring_units_fraction = distance_neighboring_units * self.layer_distance_ratio
        distance_corner_units = distance_matrix[(self.dimension[0] - 1, self.dimension[1] - 1)]
        #layer_distances = np.linspace(distance_neighboring_units_fraction, distance_corner_units, self.num_layers - 1)
        layer_distances = dist
        layer_distances = np.insert(layer_distances, 0, 1.0)  # distence to layer itself
        return layer_distances

    # create a weights matrix for two concepts in a feature matrix
    # the shape corresponds to shape (num_layers, input_len))
    # where num_soms is the number of soms trained
    def _create_weights_by_layer(self):
        if self.concept_indices.shape[0] != self._input_len:
            raise AttributeError('concept_indices has to have the same dimension as input_len')
        column_weights = []
        weights_concept_1 = np.linspace(0, 1, self.num_layers)
        weights_concept_2 = np.linspace(1, 0, self.num_layers)
        for i in self.concept_indices:
            if i:
                column_weights.append(weights_concept_1)
            else:
                column_weights.append(weights_concept_2)
        return np.column_stack(column_weights)

    # initialize all layers of the aligned SOM
    def _create_layers(self) -> List[Layer]:
        layers = []
        if self._initial_codebook_inizialization == 'random':
            inital_weights = self._create_random_weights()
        elif self._initial_codebook_inizialization == 'pretrained':
            inital_weights = self._create_weights_by_training_one_some()
        else:
            raise AttributeError('initial_codebook_inizialization has to be "random" or "pretrained"')
        for weights in self.weights_by_layer:
            layers.append(Layer(
                dimension=self.dimension,
                input_len=self._input_len,
                initial_codebook=np.array(inital_weights * weights, dtype=np.float32),
                sigma=self._sigma,
                learning_rate=self._learning_rate,
                neighborhood_function=self._neighborhood_function,
                activation_distance=self._activation_distance,
                random_seed=self.random_seed))
        return layers

    def _create_random_weights(self):
        if self.random_seed:
            np.random.seed(self.random_seed)
        return np.random.random((self.dimension[0], self.dimension[1], self._input_len))

    def _create_weights_by_training_one_some(self):
        # som trained on not weighted features (same as middle layer)
        middle_som = MiniSom(
            x=self.dimension[0],
            y=self.dimension[1],
            input_len=self._input_len,
            sigma=self._sigma,
            learning_rate=self._learning_rate,
            neighborhood_function=self._neighborhood_function,
            topology='rectangular',
            activation_distance=self._activation_distance,
            random_seed=self.random_seed)
        middle_som.train(self.data, 1000)
        return middle_som.get_weights()


In [ ]:
SEED = 12345
N_LAYERS = 31
SOM_DIM = (3, 4)
TRAIN_STEPS = 1000

input_data, components, weights, classinfo = load_dataset('animals')
data = input_data['arr']
concept_indices = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0])
concept_indices = np.abs(concept_indices - 1)

In [ ]:
asom = AlignedSom(
    SOM_DIM, data, concept_indices,
    num_layers=N_LAYERS,
    sigma=1.0,
    initial_codebook_inizialization='random')

In [ ]:
asom.layer_distances

In [ ]:
asom.train(data, TRAIN_STEPS * N_LAYERS)

In [ ]:
#SDH - implementation
def SDH(_m, _n, _weights, _idata, factor, approach):
    import heapq

    sdh_m = np.zeros( _m * _n)

    cs=0
    for i in range(factor): cs += factor-i

    for vector in _idata:
        dist = np.sqrt(np.sum(np.power(_weights - vector, 2), axis=1))
        c = heapq.nsmallest(factor, range(len(dist)), key=dist.__getitem__)
        if (approach==0): # normalized
            for j in range(factor):  sdh_m[c[j]] += (factor-j)/cs 
        if (approach==1):# based on distance
            for j in range(factor): sdh_m[c[j]] += 1.0/dist[c[j]] 
        if (approach==2): 
            dmin, dmax = min(dist[c]), max(dist[c])
            for j in range(factor): sdh_m[c[j]] += 1.0 - (dist[c[j]]-dmin)/(dmax-dmin)
    
    sdh_m = sdh_m.reshape(_m, _n)
    return resize(sdh_m, (1000, 1000), mode='constant')


In [ ]:
som_dimension = SOM_DIM
visualizations = []
for layer_weights in asom.get_layer_weights():
    layer_weights = np.reshape(layer_weights, (som_dimension[0] * som_dimension[1], data.shape[1]))
    # sdh = hv.Image(SDH(som_dimension[0], som_dimension[1], layer_weights, data, 2, 0)).opts(xaxis=None, yaxis=None)
    sdh = SDH(som_dimension[0], som_dimension[1], layer_weights, data, 2, 2)
    visualizations.append(sdh)

In [ ]:
figure, axis = plt.subplots(1, 5, figsize=(30,5))
for i, vis_i in enumerate(np.linspace(0, N_LAYERS - 1, 5, dtype=int)):
    sns.heatmap(visualizations[vis_i], ax=axis[i])
plt.show()

In [ ]:
import holoviews as hv
import panel as pn
hv.extension('bokeh')

In [ ]:
som_dimension = SOM_DIM
visualizations = []
for layer_weights in asom.get_layer_weights():
    layer_weights = np.reshape(layer_weights, (som_dimension[0] * som_dimension[1], data.shape[1]))
    # sdh = hv.Image(SDH(som_dimension[0], som_dimension[1], layer_weights, data, 2, 0)).opts(xaxis=None, yaxis=None)
    sdh = hv.Image(SDH(SOM_DIM[0], SOM_DIM[1], layer_weights, data, 2, 2)).opts(xaxis=None, yaxis=None)
    visualizations.append(sdh)

In [ ]:
pn.Row(*visualizations)

In [ ]:
sdh = hv.Image(SDH(SOM_DIM[0], SOM_DIM[1], w, data, 2, 0)).opts(xaxis=None, yaxis=None)
sdh

In [ ]:
w = asom.get_layer_weights()[0]
w = np.reshape(w,(3*4,13))

In [ ]:
somoclu.Somoclu(SOM_DIM[0], SOM_DIM[1], initialcodebook=w).view_umatrix()

In [ ]:
PySOMVis(weights_list=asom.get_layer_weights(), input_data=data)._mainview

In [ ]:
visualizations = []
layer_weights = asom.get_layer_weights()
for i in np.linspace(0, N_LAYERS - 1, 5, dtype=int):
    visualizations.append(PySOMVis(weights=layer_weights[i], input_data=data)._mainview)

In [ ]:
pn.Row(*visualizations)

In [ ]:
assert True == False

In [ ]:
neigx = np.arange(5)
neigy = np.arange(5)
neigz = np.arange(5) * 10

In [ ]:
a, b, c = np.meshgrid(neigx, neigy, neigz)

In [ ]:
X = np.array([a.reshape(1, -1)[0], b.reshape(1, -1)[0], c.reshape(1, -1)[0]])
X

In [ ]:
a.reshape(1, -1)

In [ ]:
b.reshape(1, -1)

In [ ]:
c.reshape(1, -1)

In [ ]:
xx

In [ ]:
yy